<a href="https://colab.research.google.com/github/sahar7888/NLP_LLM_Projects/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Back Translation**



In [1]:
# !pip install sentencepiece
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [2]:
#%% Model and Tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [3]:
# %% from original to target language
original_en = 'The pen is mightier than the sword.'
original_encoded = tokenizer(original_en, return_tensors="pt")
generated_tokens = model.generate(**original_encoded, forced_bos_token_id=tokenizer.get_lang_id("de"))
target_de = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
target_de

['Der Pen ist stärker als das Schwert.']

In [4]:
# %% from target back to original language
target_encoding = tokenizer(target_de, return_tensors="pt", padding=True)
generated_tokens = model.generate(**target_encoding, forced_bos_token_id=tokenizer.get_lang_id("en"))

tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['The pen is stronger than the sword.']

# **Replace with Synonyms**

In [5]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.4 MB/s eta 0:00:00


In [6]:
#%% packages
import nlpaug.augmenter.word as naw

# %%
text = "The quick brown fox jumps over the lazy dog."
aug = naw.SynonymAug(aug_src='wordnet')
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Original:
The quick brown fox jumps over the lazy dog.
Augmented Text:
['The quick john brown fox jumps complete the slothful dog.']


# **Random Crop**

In [7]:
#%% packages
import nlpaug.augmenter.word as naw

# %%
text = "The quick brown fox jumps over the lazy dog."
aug = naw.RandomWordAug(action="crop")
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
The quick brown fox jumps over the lazy dog.
Augmented Text:
['The quick brown fox jumps over the']


# **Contextual Augmentation**

In [8]:
#%% packages
import nlpaug.augmenter.word as naw

# %%
model_name = 'bert-base-uncased'
augmentation = naw.ContextualWordEmbsAug(model_path=model_name, action="substitute", device='cuda')
text_original = 'The pen is mightier than the sword.'
augmentation.augment(text_original)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

['the answer is mightier than anything machine.']

# **Fill Mask**

In [10]:
#%%
from transformers import pipeline

#%%
pipe = pipeline('fill-mask')
pipe('The pen is <mask> than the sword.')

No model was supplied, defaulted to distilbert/distilroberta-base and revision ec58a5b (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'score': 0.2354174703359604,
  'token': 3651,
  'token_str': ' stronger',
  'sequence': 'The pen is stronger than the sword.'},
 {'score': 0.1452086716890335,
  'token': 33997,
  'token_str': ' thicker',
  'sequence': 'The pen is thicker than the sword.'},
 {'score': 0.13846582174301147,
  'token': 2671,
  'token_str': ' bigger',
  'sequence': 'The pen is bigger than the sword.'},
 {'score': 0.09666416794061661,
  'token': 357,
  'token_str': ' better',
  'sequence': 'The pen is better than the sword.'},
 {'score': 0.04985335096716881,
  'token': 25731,
  'token_str': ' louder',
  'sequence': 'The pen is louder than the sword.'}]